In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import shutil
import requests
from bs4 import BeautifulSoup
from password import passwordTrier
import warnings
warnings.filterwarnings("ignore")
from functions import *

In [2]:
link_ip = '187.112.207.164'

In [3]:
df_cadastra_depto = pd.read_excel('rx_com_precos.xlsx')
df_cadastra_depto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3829 entries, 0 to 3828
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ean                  3829 non-null   int64  
 1   produto              3829 non-null   object 
 2   laboratorio          3829 non-null   object 
 3   codshopfarma         3829 non-null   int64  
 4   departamento         3693 non-null   float64
 5   preco_compra         3829 non-null   float64
 6   margem               3829 non-null   float64
 7   preco_shopfarma      3829 non-null   float64
 8   preco_preco_popular  2749 non-null   float64
 9   preco_raia           3441 non-null   float64
 10  preco_paguemenos     2069 non-null   float64
 11  preco_saojoao        2375 non-null   float64
 12  preco_medio          3612 non-null   float64
 13  preco_minimo         3829 non-null   float64
 14  preco_min_medio      3829 non-null   float64
 15  preco_novo           3829 non-null   f

In [4]:
df = df_cadastra_depto.dropna(subset=['departamento'])
df['departamento'] = df['departamento'].astype(int)
df = df.reset_index(drop=True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3693 entries, 0 to 3692
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ean                  3693 non-null   int64  
 1   produto              3693 non-null   object 
 2   laboratorio          3693 non-null   object 
 3   codshopfarma         3693 non-null   int64  
 4   departamento         3693 non-null   int32  
 5   preco_compra         3693 non-null   float64
 6   margem               3693 non-null   float64
 7   preco_shopfarma      3693 non-null   float64
 8   preco_preco_popular  2744 non-null   float64
 9   preco_raia           3374 non-null   float64
 10  preco_paguemenos     2068 non-null   float64
 11  preco_saojoao        2373 non-null   float64
 12  preco_medio          3544 non-null   float64
 13  preco_minimo         3693 non-null   float64
 14  preco_min_medio      3693 non-null   float64
 15  preco_novo           3693 non-null   f

,ean,produto,laboratorio,codshopfarma,departamento,preco_compra,margem,preco_shopfarma,preco_preco_popular,preco_raia,preco_paguemenos,preco_saojoao,preco_medio,preco_minimo,preco_min_medio,preco_novo,margem_nova
0,7891317129941,20 BI 335MG 10CAP,MOMENTA,18717,5,66.69,31.938821,87.99,82.95,83.73,NaN,74.39,80.356667,74.39,77.373333,76.99,15.444594
1,7891317129958,20 BI 335MG 30CAP,MOMENTA,122060,5,132.28,37.511340,181.90,207.95,187.99,NaN,175.59,190.510000,175.59,183.050000,180.99,36.823405
2,7891317019297,20 BI 335MG 5CAP,MOMENTA,54135,5,43.18,38.930060,59.99,NaN,55.83,55.25,NaN,55.540000,55.25,55.395000,54.99,27.350625
3,7896004817477,A SAUDE DA MULHER LIQ 150ML,EMS,107541,5,11.90,38.655462,16.50,17.77,15.39,19.15,18.51,17.705000,15.39,16.547500,15.99,34.369748
4,7897595901927,AAS 100MG 120CP,HYPERA,104826,5,54.48,51.964023,82.79,73.74,73.99,79.35,65.69,73.192500,65.69,69.441250,68.99,26.633627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3688,7897316805688,ZYMAR XD 5MG/ML COL 5ML,ALLERGAN,109701,1,49.40,72.024291,84.98,69.21,66.19,NaN,80.46,71.953333,66.19,69.071667,68.99,39.655870
3689,7897316805619,ZYPRED 3MG+10MG/ML COL 3ML,ALLERGAN,15657,1,28.26,54.777070,43.74,37.35,36.39,NaN,41.42,38.386667,36.39,37.388333,36.99,30.891720
3690,7897316805442,ZYPRED 3MG+10MG/ML COL 6ML,ALLERGAN,111924,1,51.75,69.120773,87.52,73.94,72.99,NaN,82.88,76.603333,72.99,74.796667,73.99,42.975845
3691,7896269901898,ZYRTEC 10MG 12CP REV,GSK,102653,1,0.01,825400.000000,82.55,69.93,67.79,74.65,NaN,70.790000,67.79,69.290000,68.99,689800.000000


In [5]:
cadastro_depto(df, col_depto='departamento')

100%|██████████| 3693/3693 [3:15:21<00:00,  3.17s/it]  
